In [1]:
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torchsummary import summary

In [2]:
EPOCHS = 20
BATCH_SIZE = 128
LEARNING_RATE = 0.001

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load data

In [3]:
train_transforms = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.4914, 0.4822, 0.4465],
                std=[0.2023, 0.1994, 0.2010])
])

val_transforms = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.4914, 0.4822, 0.4465],
                std=[0.2023, 0.1994, 0.2010])
])

In [4]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transforms)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=val_transforms)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


# Init model

In [5]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.block1 = self._make_conv_block(in_ch=3,   out_ch=64)
        self.block2 = self._make_conv_block(in_ch=64,  out_ch=128)
        self.block3 = self._make_conv_block(in_ch=128, out_ch=256)
        self.block4 = self._make_conv_block(in_ch=256, out_ch=512)

        self.classifier = nn.Sequential(
            nn.Linear(512 * 2 * 2, 2048),  # из 512x2x2 -> 2048
            nn.ReLU(inplace=True),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_classes)
        )

    def _make_conv_block(self, in_ch, out_ch):
        block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(kernel_size=2)
        )
        return block

    def forward(self, x):
        x = self.block1(x) 
        x = self.block2(x)  
        x = self.block3(x)  
        x = self.block4(x) 

        x = x.view(x.size(0), -1)  
        x = self.classifier(x)
        return x

model = CNN(num_classes=10).to(DEVICE)

_ = summary(model, input_size=(BATCH_SIZE, 3, 32, 32), device=DEVICE, depth=4)

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       1,728
|    └─BatchNorm2d: 2-2                  128
|    └─ReLU: 2-3                         --
|    └─Conv2d: 2-4                       36,864
|    └─BatchNorm2d: 2-5                  128
|    └─ReLU: 2-6                         --
|    └─MaxPool2d: 2-7                    --
├─Sequential: 1-2                        --
|    └─Conv2d: 2-8                       73,728
|    └─BatchNorm2d: 2-9                  256
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      147,456
|    └─BatchNorm2d: 2-12                 256
|    └─ReLU: 2-13                        --
|    └─MaxPool2d: 2-14                   --
├─Sequential: 1-3                        --
|    └─Conv2d: 2-15                      294,912
|    └─BatchNorm2d: 2-16                 512
|    └─ReLU: 2-17                        --
|    └─Conv2d: 2-18                      589,

# Train

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [7]:
def train_one_epoch(model, dataloader, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for X_, y_ in dataloader:
        X_, y_ = X_.to(DEVICE), y_.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(X_)
        loss = criterion(outputs, y_)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_).sum().item()
        total += y_.size(0)

    epoch_loss = train_loss / len(dataloader)
    accuracy = 100.0 * correct / total
    return epoch_loss, accuracy

def evaluate(model, dataloader):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for X_, y_ in dataloader:
            X_, y_ = X_.to(DEVICE), y_.to(DEVICE)

            outputs = model(X_)
            loss = criterion(outputs, y_)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_).sum().item()
            total += y_.size(0)

    val_loss = val_loss / len(dataloader)
    val_acc = 100.0 * correct / total
    return val_loss, val_acc

In [8]:
for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer)
    val_loss, val_acc = evaluate(model, val_loader)

    print(f"Epoch [{epoch+1}/{EPOCHS}]")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")

Epoch [1/20]
Train Loss: 1.7982 | Train Acc: 30.37%
Val Loss:   1.4346 | Val Acc:   46.42%
Epoch [2/20]
Train Loss: 1.2190 | Train Acc: 55.24%
Val Loss:   1.2493 | Val Acc:   57.21%
Epoch [3/20]
Train Loss: 0.9197 | Train Acc: 67.42%
Val Loss:   0.9069 | Val Acc:   67.90%
Epoch [4/20]
Train Loss: 0.7667 | Train Acc: 73.13%
Val Loss:   0.8474 | Val Acc:   70.41%
Epoch [5/20]
Train Loss: 0.6426 | Train Acc: 77.73%
Val Loss:   0.6772 | Val Acc:   76.86%
Epoch [6/20]
Train Loss: 0.5702 | Train Acc: 80.52%
Val Loss:   0.6349 | Val Acc:   79.50%
Epoch [7/20]
Train Loss: 0.5104 | Train Acc: 82.56%
Val Loss:   0.5623 | Val Acc:   80.51%
Epoch [8/20]
Train Loss: 0.4607 | Train Acc: 84.20%
Val Loss:   0.5302 | Val Acc:   82.29%
Epoch [9/20]
Train Loss: 0.4225 | Train Acc: 85.43%
Val Loss:   0.5109 | Val Acc:   83.05%
Epoch [10/20]
Train Loss: 0.3895 | Train Acc: 86.62%
Val Loss:   0.4999 | Val Acc:   83.37%
Epoch [11/20]
Train Loss: 0.3627 | Train Acc: 87.71%
Val Loss:   0.4620 | Val Acc:   85.1

# Eval

In [15]:
inp = torch.randn(1, 3, 32, 32)

num_samples = 100
start_time = time.time()
for _ in range(num_samples):
    output = model(inp.to(DEVICE))
end_time = time.time()

infer_time = ((end_time - start_time) / num_samples) * 1000
print(f'Avg inference time: {infer_time:.4f} ms')

Avg inference time: 0.5625 ms


In [10]:
model_size_mb = (sum(p.numel() for p in model.parameters()) * 4) / (1024**2)  # float32
print(f"Model size: {model_size_mb:.2f} MB")

Model size: 41.93 MB


In [11]:
torch.save(model.state_dict(), './model.pt')